<a href="https://colab.research.google.com/github/AlanMarket/FirstRepository/blob/master/10_29_VIF_%26_SKlearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import seaborn as sns

import statsmodels.api as sm
import sklearn as sk

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LassoCV, LarsCV, LassoLarsCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score

from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
excelfile = 'drive/My Drive/ColabFiles/nativeads.xls'
adataframe = pd.read_excel(excelfile)

In [0]:
pivoteddata = adataframe.pivot_table(index='URL', aggfunc='mean')

In [0]:
pivoteddata.head()

,CPA,CTR(%),Clicks,Conversions,Cost,Impressions
URL,,,,,,
http://aluxurytravelblog.com/2013/10/23/4-simple-steps-to-cheaper-hotel-rates-with-travelpony/,3.180000,0.025300,28.000000,2.000000,6.360000,1.105280e+05
http://beersandbeans.com/2013/10/14/travelpony-review/,1.292945,0.041042,1222.846154,65.653846,240.895839,1.078494e+06
http://beinginawe.com/how-booking-the-hotel-with-travelpony-com-saved-us-a-lot-of-time-and-money/,0.141167,0.177659,68.242593,1.570370,5.578093,3.626747e+04
http://blog.travelpony.com/2013/09/25/5-must-see-attractions-in-atlanta/,0.000000,0.067500,93.333333,0.000000,19.123333,1.192400e+05
http://blog.travelpony.com/2013/09/30/7-of-the-best-free-attractions-in-boston/,0.000000,0.020300,0.666667,0.000000,0.166667,3.015000e+03


In [0]:
pivoteddata.columns

Index(['CPA', 'CTR(%)', 'Clicks', 'Conversions', 'Cost', 'Impressions'], dtype='object')

In [0]:
target=pivoteddata['Conversions']
dep= pivoteddata[['CPA','CTR(%)', 'Clicks', 'Cost', 'Impressions']]
mod = sm.OLS(target, dep)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:            Conversions   R-squared (uncentered):                   0.931
Model:                            OLS   Adj. R-squared (uncentered):              0.927
Method:                 Least Squares   F-statistic:                              284.4
Date:                Wed, 13 Nov 2019   Prob (F-statistic):                    1.04e-59
Time:                        00:41:39   Log-Likelihood:                         -420.22
No. Observations:                 111   AIC:                                      850.4
Df Residuals:                     106   BIC:                                      864.0
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [0]:
predictions = ols.predict(dep)
error = abs(predictions-target)
meanerror = np.mean(error)
print(meanerror)
print(np.std(error))

4.993822779306258
9.422132886565878


In [0]:
for predictor in range(len(dep.columns)):
  print(predictor)
  vifscore = variance_inflation_factor(dep.values, predictor)
  print(dep.columns[predictor], vifscore)


0
CPA 1.2889209626296867
1
CTR(%) 1.3486620305946422
2
Clicks 21.246456657147966
3
Cost 42.345934675263685
4
Impressions 12.385984492641153


In [0]:
model = LassoLarsCV(cv=20, precompute=False)
lasso = model.fit(dep, target)


In [0]:
predictors_model=pd.DataFrame(dep.columns)
predictors_model.columns = ['label']
predictors_model['coeff'] = model.coef_

In [0]:
predictors_model

,label,coeff
0,CPA,-0.193472
1,CTR(%),0.000000
2,Clicks,0.000000
3,Cost,0.088407
4,Impressions,0.000018


In [0]:
predictions = lasso.predict(dep)
error = abs(predictions-target)
meanerror = np.mean(error)
print(meanerror)
print(np.std(error))

5.287356599892492
9.796610892823637


In [0]:
lars = sk.linear_model.Lars().fit(dep, target)

#lassoLars = sk.linear_model.LassoLarsCV().fit(dep, target)

In [0]:
columns = ['Test RMSE', 'Test R2']
rows = ['Lars','Lasso', 'ols', 'LassoCV']
results = pd.DataFrame(columns=columns, index=rows)

methods = [lars,lasso,ols,lassoCV]
# .iloc[row, column]
for i, method in enumerate(methods):
  y_pred = method.predict(dep)
  results.iloc[i,0] = np.sqrt(mean_squared_error(target,y_pred))
  results.iloc[i,1] = r2_score(target, y_pred)

NameError: ignored

In [0]:
results

,Test RMSE,Test R2
Lars,10.6637,0.925156
Lasso,11.1324,0.918433
ols,10.6637,0.925156


In [0]:
predictions = lars.predict(dep)
error = abs(predictions-target)
meanerror = np.mean(error)
print(meanerror)
print(np.std(error))

4.989450647003898
9.424441518169074


In [0]:
model = LassoCV(cv=20, precompute=False)
lasso = model.fit(dep, target)

In [0]:
predictors_model=pd.DataFrame(dep.columns)
predictors_model.columns = ['label']
predictors_model['coeff'] = lars.coef_
predictors_model

,label,coeff
0,CPA,-0.274662
1,CTR(%),11.013973
2,Clicks,-0.007647
3,Cost,0.148190
4,Impressions,0.000015
